https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
# !git clone https://huggingface.co/qarib/bert-base-qarib

     |████████████████████████████████| 258 kB 6.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 10.9 MB/s 
     |████████████████████████████████| 164 kB 66.2 MB/s 
     |████████████████████████████████| 75 kB 5.4 MB/s 
     |████████████████████████████████| 141 kB 75.1 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
     |████████████████████████████████| 111 kB 75.4 MB/s 
     |████████████████████████████████| 1.8 MB 7.9 MB/s 
     |████████████████████████████████| 2.9 MB 55.9 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 126 kB 8.3 MB/s 


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [4]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/PPV2train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/PPV2val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']


# Trainer

In [5]:
# from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [6]:
#https://huggingface.co/qarib/bert-base-qarib60_1970k
dataset_name = 'NEWS'
model_name = 'qarib/bert-base-qarib60_1970k'
task_name = 'classification'
max_len = 414

In [7]:
# arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

# df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
# df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [8]:
class BERTDataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

# 

In [9]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()  ,df_val.label.to_list()  ,model_name,  max_len, label_map)


{'fake': 0, 'real': 1}


In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [11]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# Regular Training

In [12]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8

steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.load_best_model_at_end=True

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [13]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

Some weights of the model checkpoint at qarib/bert-base-qarib60_1970k were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.354900,0.800001,0.917213,0.759336,0.869735,16.742400,76.572000
1,No log,0.251704,0.931429,0.953630,0.913968,0.948518,17.030900,75.275000
2,No log,0.184008,0.953256,0.963619,0.944048,0.964119,17.036800,75.249000
3,0.124200,0.182585,0.963201,0.977105,0.951223,0.971919,17.005500,75.387000
4,0.124200,0.178610,0.967609,0.976169,0.959829,0.975039,17.009300,75.370000
5,0.124200,0.197854,0.966367,0.978669,0.955596,0.974259,17.019500,75.325000
6,0.004200,0.201810,0.965314,0.978147,0.954138,0.973479,16.993100,75.442000
7,0.004200,0.202057,0.965314,0.978147,0.954138,0.973479,17.025400,75.299000


              precision    recall  f1-score   support

           0       0.98      0.52      0.68       343
           1       0.85      1.00      0.92       939

    accuracy                           0.87      1282
   macro avg       0.92      0.76      0.80      1282
weighted avg       0.89      0.87      0.85      1282

[[179 164]
 [  3 936]]
              precision    recall  f1-score   support

           0       0.96      0.84      0.90       343
           1       0.94      0.99      0.97       939

    accuracy                           0.95      1282
   macro avg       0.95      0.91      0.93      1282
weighted avg       0.95      0.95      0.95      1282

[[288  55]
 [ 11 928]]
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       343
           1       0.96      0.99      0.98       939

    accuracy                           0.96      1282
   macro avg       0.96      0.94      0.95      1282
weighted avg       0.96     

TrainOutput(global_step=1280, training_loss=0.05014058003143873, metrics={'train_runtime': 1666.2339, 'train_samples_per_second': 0.768, 'total_flos': 13774160747798208, 'epoch': 8.0})

In [15]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.99      0.91      0.95       343
           1       0.97      1.00      0.98       939

    accuracy                           0.97      1282
   macro avg       0.98      0.95      0.97      1282
weighted avg       0.97      0.97      0.97      1282

[[313  30]
 [  4 935]]


{'epoch': 8.0,
 'eval_accuracy': 0.9734789391575663,
 'eval_loss': 0.20205652713775635,
 'eval_macro_f1': 0.9653138528138527,
 'eval_macro_precision': 0.9781468102842386,
 'eval_macro_recall': 0.9541382961217348,
 'eval_runtime': 16.9869,
 'eval_samples_per_second': 75.47}

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
trainer._save("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB")
tokenizer.save_pretrained("/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB")

('/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/tokenizer_config.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/special_tokens_map.json',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/vocab.txt',
 '/content/drive/MyDrive/Arabic_FakeNews/Model/Balance/SavedModels/QaRiB/added_tokens.json')

# Test

In [18]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataDescr/PPV2test_news.csv")
print(df_test.label.value_counts())


1    1203
0     400
Name: label, dtype: int64


In [19]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])

In [20]:
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)

In [21]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       0.96      0.89      0.93       400
           1       0.97      0.99      0.98      1203

    accuracy                           0.97      1603
   macro avg       0.96      0.94      0.95      1603
weighted avg       0.97      0.97      0.96      1603

[[ 357   43]
 [  13 1190]]


{'epoch': 8.0,
 'eval_accuracy': 0.9650655021834061,
 'eval_loss': 0.2575729787349701,
 'eval_macro_f1': 0.9521421107628004,
 'eval_macro_precision': 0.9649952872580609,
 'eval_macro_recall': 0.9408468412302577,
 'eval_runtime': 21.0013,
 'eval_samples_per_second': 76.329}

# Predict

In [22]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [23]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 4


In [24]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [25]:
# # Tokenize test data
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512) 
# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized) 
# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2) 
# # Define test trainer
# test_trainer = Trainer(model) 
# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset) 
# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [26]:
# print("===> Predicting...")
# trainer1 = Trainer(
#     model= model_init(),
#     args=training_args,
#     eval_dataset=test_dataset
# )
        
# results = {}
# result = trainer1.predict(test_dataset)

In [27]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)